In [5]:
import sys
sys.executable


'C:\\Users\\suzal\\AppData\\Local\\Programs\\Python\\Python313\\python.exe'

In [6]:
!pip install pandas numpy scikit-learn xgboost


In [1]:
import pandas as pd
import numpy as np
import sklearn
import xgboost

print("ALL OK")




ALL OK


In [67]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier


In [69]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

df_fake["label"] = 0
df_true["label"] = 1

df = pd.concat([df_fake, df_true], axis=0)
df = df.sample(frac=1).reset_index(drop=True)
df = df[["text", "label"]]


In [70]:
df_fake.head(), df_true.head()


(                                               title  \
 0   Donald Trump Sends Out Embarrassing New Year’...   
 1   Drunk Bragging Trump Staffer Started Russian ...   
 2   Sheriff David Clarke Becomes An Internet Joke...   
 3   Trump Is So Obsessed He Even Has Obama’s Name...   
 4   Pope Francis Just Called Out Donald Trump Dur...   
 
                                                 text subject  \
 0  Donald Trump just couldn t wish all Americans ...    News   
 1  House Intelligence Committee Chairman Devin Nu...    News   
 2  On Friday, it was revealed that former Milwauk...    News   
 3  On Christmas day, Donald Trump announced that ...    News   
 4  Pope Francis used his annual Christmas Day mes...    News   
 
                 date  label  
 0  December 31, 2017      0  
 1  December 31, 2017      0  
 2  December 30, 2017      0  
 3  December 29, 2017      0  
 4  December 25, 2017      0  ,
                                                title  \
 0  As U.S. budget f

In [73]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

df["text"] = df["text"].apply(clean_text)



In [74]:
X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [82]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM": LinearSVC(),
    "XGBoost": XGBClassifier(eval_metric="logloss")
}



In [84]:
accuracies = {}

for name, model in models.items():
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)

    acc = accuracy_score(y_test, y_pred)
    accuracies[name] = acc

    print("="*50)
    print(name)
    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred))



Logistic Regression
Accuracy: 0.9837861915367483
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      5846
           1       0.98      0.98      0.98      5379

    accuracy                           0.98     11225
   macro avg       0.98      0.98      0.98     11225
weighted avg       0.98      0.98      0.98     11225

Decision Tree
Accuracy: 0.9946547884187082
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5846
           1       0.99      0.99      0.99      5379

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

Random Forest
Accuracy: 0.9962583518930958
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5846
           1       1.00      1.00      1.00      5379

    accuracy                           1.00    

In [90]:
best_model_name = max(accuracies, key=accuracies.get)
best_model = models[best_model_name]

print("Best Model Selected:", best_model_name)

joblib.dump(best_model, "best_fake_news_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

print("Model & Vectorizer Saved")



Best Model Selected: XGBoost
Model & Vectorizer Saved


In [92]:
def predict_news(news):
    news = clean_text(news)
    vec = vectorizer.transform([news])

    print("\nPrediction Results:\n")
    for name, model in models.items():
        pred = model.predict(vec)[0]
        print(f"{name}: {'Real News' if pred == 1 else 'Fake News'}")


In [79]:
news = input("Enter news text: ")
predict_news(news)


Enter news text:  Trump signs tax, government spending bills into law



Prediction Results:

Logistic Regression: Real News
Decision Tree: Fake News
Random Forest: Fake News
Gradient Boosting: Fake News
SVM: Real News
XGBoost: Fake News
